In [ ]:
import torch
from torch.nn import functional as F 
import tqdm
import numpy as np

def net_training(net, train_loader, test_loader, criterion, learning_rate, N, train_MSE=None, test_MSE=None):
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    if train_MSE is None:
        train_MSE = []
    if test_MSE is None:
        test_MSE = []
    for iter in tqdm.tqdm_notebook(range(N)):
        epoch_MSE = 0.0
        epoch_iter = 0
        error = []
        for item in train_loader:

            inputs = torch.tensor(item[0], dtype=torch.float32)
            labels = torch.tensor(item[1], dtype=torch.float32)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs.squeeze(1), labels)
            loss.backward()
            optimizer.step()

            epoch_MSE += F.mse_loss(outputs.squeeze(1), labels)
            epoch_iter += 1
        errors = 0.
        for test_item in test_loader:
            inputs = torch.tensor(test_item[0], dtype=torch.float32)
            labels = torch.tensor(test_item[1], dtype=torch.float32)
            outputs = net(inputs)
            errors += F.mse_loss(outputs.squeeze(1), labels)
        test_MSE.append(errors.data)
        epoch_MSE /= epoch_iter
        train_MSE.append(epoch_MSE.data)
    return train_MSE, test_MSE

def net_cuda_training(net, train_loader, test_loader, criterion, learning_rate, N, train_MSE=None, test_MSE=None):
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    if train_MSE is None:
        train_MSE = []
    if test_MSE is None:
        test_MSE = []
    for iter in tqdm.tqdm_notebook(range(N)):
        epoch_MSE = 0.0
        epoch_iter = 0
        error = []
        for item in train_loader:

            inputs = item[0].type(torch.cuda.FloatTensor)
            labels = item[1].type(torch.cuda.FloatTensor)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs.squeeze(1), labels)
            loss.backward()
            optimizer.step()

            epoch_MSE += F.mse_loss(outputs.squeeze(1), labels)
            epoch_iter += 1
        errors = 0.
        for test_item in test_loader:
            inputs = item[0].type(torch.cuda.FloatTensor)
            labels = item[1].type(torch.cuda.FloatTensor)
            outputs = net(inputs)
            errors += F.mse_loss(outputs.squeeze(1), labels)
        test_MSE.append(errors.data)
        epoch_MSE /= epoch_iter
        train_MSE.append(epoch_MSE.data)
    return train_MSE, test_MSE

def whole_pic_lines(X):
    L = int((X.shape[0] * X.shape[1]) ** 0.5)
    picture = np.zeros((L, L))
    d = int((X.shape[1]) ** 0.5)
    k = 0
    for i in range(0, L, d):
        for j in range(0, L, d):
            picture[i: i + d, j: j + d] = np.reshape(X[k], (d, d))
            k += 1
    return picture

def whole_pic(X):
    L = int((X.shape[0] * X.shape[1]) ** 0.5)
    picture = np.zeros((L, L))
    d = int((X.shape[1]) ** 0.5)
    k = 0
    for i in range(0, L, d):
        for j in range(0, L, d):
            picture[i: i + d, j: j + d] = np.reshape(X[k], (d, d))
            k += 1
    return picture